# Model Evaluation Script

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers, optimizers
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from keras import backend as K
from sklearn.metrics import log_loss


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../March-Madness"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


Data
Data_Organization.ipynb
Final
Final_Data_Organization.ipynb
Final MLP.ipynb
GiantKillerInfo.txt
logs
MarchMadnessAdvStats.csv
MarchMadnessFeatureDifferences.csv
MarchMadnessFeatures.csv
MarchMadnessTest.csv
MLP Make Bracket.ipynb
ModelEvaluation.ipynb
OldNbs
README.md
SubmissionData.py
Submissions
tENSORFLOW.ipynb
tENSORFLOW.py
tENSORFLOW-TB.ipynb



## Load pre-organized data

In [2]:
data_dir = '../March-Madness/'
df_features = pd.read_csv(data_dir + 'MarchMadnessFeatures.csv')

Format and scale the data

In [64]:
X = df_features.iloc[:,1:]
xDim = np.shape(X)[1]
X_train = X.values.reshape(-1,xDim)
y_train = df_features.Result.values

print('Feature vector dimension is: %.2f' % xDim)
# print(X[:5, :])

Feature vector dimension is: 34.00


In [65]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_train = pca.fit_transform(X_train)

In [66]:
xDim = np.shape(X_train)[1]

# Load Training Data


In [67]:
df_test = pd.read_csv(data_dir+'MarchMadnessTest.csv')

X_test = df_test.iloc[:,1:]
xDimTest = np.shape(X_test)[1]
X_test = X_test.values.reshape(-1,xDimTest)
y_test = df_test.Result.values

X_test = pca.transform(X_test)

Create Kfold splits

In [68]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

In [69]:
np.random.seed()

## MLP

Various properties

In [70]:
dropRate = 0.3
numBatch = 50
numEpoch = 120
learningRate = 1e-4

Base Single Layer Model

In [94]:
# MLP model
MLP = Sequential()
MLP.name = 'MLP'
MLP.add(Dense(20, input_dim=xDim, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(200,kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(200, kernel_initializer='glorot_normal',activation = 'tanh'))
MLP.add(Dropout(dropRate))
MLP.add(Dense(1, kernel_initializer='glorot_normal', activation='sigmoid'))

# Compile model
adam = optimizers.Adam(lr=learningRate, amsgrad=True)
sgd = optimizers.SGD(lr=learningRate)
MLP.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])

In [95]:
MLP.fit(X_train, y_train, epochs=numEpoch, batch_size=numBatch, verbose=1)

Epoch 1/120
530/530 [==============================] - 1s 1ms/step - loss: 0.2486 - acc: 0.5604
Epoch 2/120
530/530 [==============================] - 0s 60us/step - loss: 0.2299 - acc: 0.6170
Epoch 3/120
530/530 [==============================] - 0s 54us/step - loss: 0.2257 - acc: 0.6509
Epoch 4/120
530/530 [==============================] - 0s 52us/step - loss: 0.2240 - acc: 0.6226
Epoch 5/120
530/530 [==============================] - 0s 57us/step - loss: 0.2210 - acc: 0.6547
Epoch 6/120
530/530 [==============================] - 0s 56us/step - loss: 0.2269 - acc: 0.6453
Epoch 7/120
530/530 [==============================] - 0s 60us/step - loss: 0.2200 - acc: 0.6566
Epoch 8/120
530/530 [==============================] - 0s 64us/step - loss: 0.2190 - acc: 0.6623
Epoch 9/120
530/530 [==============================] - 0s 54us/step - loss: 0.2206 - acc: 0.6717
Epoch 10/120
530/530 [==============================] - 0s 46us/step - loss: 0.2232 - acc: 0.6472
Epoch 11/120
530/530 [========

530/530 [==============================] - 0s 51us/step - loss: 0.2144 - acc: 0.6679
Epoch 85/120
530/530 [==============================] - 0s 55us/step - loss: 0.2096 - acc: 0.6792
Epoch 86/120
530/530 [==============================] - 0s 50us/step - loss: 0.2143 - acc: 0.6774
Epoch 87/120
530/530 [==============================] - 0s 49us/step - loss: 0.2163 - acc: 0.6755
Epoch 88/120
530/530 [==============================] - 0s 45us/step - loss: 0.2191 - acc: 0.6642
Epoch 89/120
530/530 [==============================] - 0s 46us/step - loss: 0.2158 - acc: 0.6623
Epoch 90/120
530/530 [==============================] - 0s 46us/step - loss: 0.2172 - acc: 0.6717
Epoch 91/120
530/530 [==============================] - 0s 49us/step - loss: 0.2144 - acc: 0.6717
Epoch 92/120
530/530 [==============================] - 0s 47us/step - loss: 0.2176 - acc: 0.6755
Epoch 93/120
530/530 [==============================] - 0s 47us/step - loss: 0.2153 - acc: 0.6698
Epoch 94/120
530/530 [===========

In [96]:
MLP.evaluate(X_test, y_test)

402/402 [==============================] - 0s 723us/step


[0.18164074183696538, 0.7462686561233368]

## Random Forest

In [ ]:
fore = RandomForestClassifier(criterion='entropy', n_estimators = 50, max_depth = 4, oob_score=True)

fore.fit(X_train, y_train)

y_pred =  fore.predict_proba(X_train)[:,1].reshape(-1,1)
LL = log_loss( y_train, y_pred)
print("Training set score: {:4}" .format(LL))
y_pred =  fore.predict_proba(X_test)[:,1].reshape(-1,1)
LL = log_loss( y_test, y_pred)
print("Validation set score: {:4}".format(LL))

In [120]:
accTest = fore.score(X_test, y_test)
print("Validation accuracy: {:4}".format(accTest))

Validation accuracy: 0.736318407960199


## Making predictions with model

### Extract data desired

In [97]:
data_dir = '../March-Madness/Final/'
df_sample_sub = pd.read_csv(data_dir + 'SampleSubmissionStage2.csv')
data_file = data_dir + 'MarchMadnessAdvStats.csv'
df_adv = pd.read_csv(data_file)
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')


n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

def seed_to_int(seed):
    '''Get just the digits from the seeding. Return as int'''
    s_int = int(seed[1:3])
    return s_int

In [98]:
print('Loading data for submission test')

# Make the seeding an integer
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(columns=['Seed'], inplace=True) # This is the string label
df_seeds.head()


T1_seed = []
T1_adv = []
T2_adv = []
T2_seed = []
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    t1_adv = df_adv[(df_adv.TeamID == t1) & (df_adv.Season == year)].values[0]
    t2_adv = df_adv[(df_adv.TeamID == t2) & (df_adv.Season == year)].values[0]
    T1_seed.append(t1_seed)
    T1_adv.append(t1_adv)
    T2_seed.append(t2_seed)
    T2_adv.append(t2_adv)

T1_adv = [row[2:] for row in T1_adv]
T2_adv = [row[2:] for row in T2_adv]
T1_seed = np.reshape(T1_seed, [n_test_games,-1]).tolist()
T2_seed = np.reshape(T2_seed, [n_test_games, -1]).tolist()
X_pred = np.concatenate((T1_seed, T1_adv, T2_seed, T2_adv), axis=1)

df_subData = pd.DataFrame(np.array(X_pred).reshape(np.shape(X_pred)[0], np.shape(X_pred)[1]))

xDim = np.shape(df_subData)[1]
X_pred = df_subData.values.reshape(-1,xDim)

Loading data for submission test


In [99]:
np.shape(X_pred)

(2278, 34)

In [100]:
X_pred = pca.transform(X_pred)

In [104]:
preds = MLP.predict(X_pred)

# df_sample_sub = pd.DataFrame()
# clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = preds
df_sample_sub.shape

(2278, 2)

In [103]:
preds = fore.predict(X_pred)

# df_sample_sub = pd.DataFrame()
# clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = preds
df_sample_sub.shape

NameError: name 'fore' is not defined

In [105]:
filename = 'makeBrack'
save_dir = '../March-Madness/Final/'
c=0
ext = '.csv'
if os.path.exists(save_dir+filename+ext):
    while os.path.exists(filename+ext):
        c+=1
    filename = filename+'_'+str(c)
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)
else:
    df_sample_sub.to_csv(save_dir+filename+ext, index=False)